# <center> A Genetic Algorithm for Solving Portfolio Optimization Problems with Transaction Costs and Minimum Transaction Lots </center>

### Name : Altaf Ahmad
### Roll no: 18MA20005 

GAs are stochastic, heuristic techniques based on the natural selection principles, and they can deal with the nonlinear optimization problems with non-smooth and even non-continuous objective, and continuous and/or integer variables.

In this paper, the authors have proposd a model for portfolio rebalancing optimization problems with transaction costs and minimum lots. A GA based on the traditional real value genetic operators is then designed to solve these models.


#### Model : 
Denote by S the set of securities to invest a capital $C_0 ≤ C ≤ C_1$ and let $s = |S|$ be the number of the securities. A portfolio can be represented as $k = ( k_1 ,..., k_s )$ , where $k_j$ represents the number of shares of security $j$ . Let $r_j$ be the random rate of return on security $j ∈ S$ . Let $n^s_j$ be the minimum transaction lot when selling security $j$ and $n^b_j$ be the minimum lot when buying security $j$ . Denote $R_j = E ( r_j )$ the expected rate of return on security $j$ and $σ_{ij} = cov( r_i , r_j )$ the covariance between $r_i$ and $r_j$ . For security $j$ , denote $u_j$ the maximum amount of capital that can be invested in it. $d_j$ represents the fixed proportion parameter of transaction cost associated with security $j$ and $p_j$ represents the quoted price, and $y_j = ( p_j k_j ) / C$ . 

The expected return $R (k )$ and the variance $V (k )$ can be written as

$$ 
R(k) = ( \sum_{j \in S} R_j p_j k_j - \sum_{j \in S} d_j p_j \big | k_j - k_j^0 \big |)/C \\
V(k) = \sum_{i \in S} \sum_{j \in S} \sigma_{ij} y_i y_j
$$

A portfolio rebalancing problem can be stated as follows:
<center>min $f(k) = - \lambda . R(k) + (1- \lambda) . \omega . V(k) $ </center> 

$$
\text{s.t.  }\;\; C_0 \leq \sum_{j \in S} p_j k_j = C \leq C_1 , \\
0 ≤ p_j k_j ≤ u_j , \; \; \;  \text{ for all } j ∈ S \\
n^b_j | ( k_j − k^0_j ), \; \; \;  \text{ for all } j ∈ S \text{, if } k_j > k^0_j , \\
n^s_j | ( k^0_j − k_j ) ,  \; \; \;  \text{ for all }j ∈ S , \text{ if } k^0_j > k_j .
$$

This means that in case buying, $k_j - k_j^0$ must be divisible by $n_j^b$, and while selling, $k_j^0 - k_j$ must be divisible by $n_j^s$. 

Here parameter $λ$ varying in $[ 0 , 1 ]$ and $ω$ is a scaling parameter.

## Algorithm : 
__*Representation Structure*__. Each portfolio in the population is coded as a string of non-negative integer numbers.

__*Repair Process*__. For an existing portfolio $k ^0 = ( k _1 ^0 ,..., k _s ^0 )$ ,a real vector $x = ( x _1 ,..., x _s )$
can be repaired into a new portfolio by the following formula:
$$
d_i=
\begin{cases}
\bigg [ \frac{x_i - k_i^0}{n_i^b} \bigg] \; \; \; \text{if}\; \; x_i \geq k_i^0\\
\bigg [ \frac{k_i^0-x_i}{n_i^s} \bigg] \; \; \; \text{if}\; \; x_i \leq k_i^0
\end{cases}
$$

$$
m_i = 
\begin{cases}
k_i ^0 + d_i ⋅ n_i ^b \; \; \; \text{ if } \; \; x_i \geq k_i^0 \\
k_i ^0 - d_i ⋅ n_i ^s \; \; \; \text{ if } \; \; x_i \leq k_i^0 
\end{cases} \\
m_i' = 
\begin{cases}
k_i ^0 + (d_i+1) ⋅ n_i ^b \; \; \; \text{ if } \; \; x_i \geq k_i^0 \\
k_i ^0 - (d_i+1) ⋅ n_i ^s \; \; \; \text{ if } \; \; x_i \leq k_i^0 
\end{cases} \\
k_i = m_i \text{ or } m_i' \text{ randomly.}
$$

&nbsp;  The GA based on the SBX and PM genetic operators can be written as follows: <br>
Step 0. &nbsp; &nbsp; Input parameters: *pop_size* , $P_c$ and $P _m$ , the parameters of SBX and PM
operators, total evolutionary generations $gen$ .<br>
Step 1. &nbsp; &nbsp; Initialize *pop_size* individuals to generate the initial population.
Step 2. &nbsp; &nbsp; Update the individuals in the current population by SBX crossover and PM
mutation operators. Repair the individuals as described in the repair
process.<br>
Step 3. &nbsp; &nbsp; Calculate the fitness values for all individuals.<br>
Step 4. &nbsp; &nbsp; Select the individuals by using the binary tournament selection strategy.<br>
Step 5. &nbsp; &nbsp; Repeat steps 2 to 4 until the given $gen$ generations.<br>

In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import glob
from itertools import islice
from pymoo.core.individual import Individual
from pymoo.core.problem import Problem
from pymoo.operators.crossover.sbx import SBX
from pymoo.core.population import Population
from pymoo.operators.mutation.pm import PolynomialMutation

Input Parameters : <br>
    &nbsp;&nbsp;&nbsp; *pop_size* = 400<br>
    &nbsp;&nbsp;&nbsp;$P_c$ = 0.95<br>
    &nbsp;&nbsp;&nbsp;$P_m$ = 1/6<br>
    

In [2]:
pop_size = 400
P_c = 0.05
P_m = 1/6
omega = 100

In [3]:
# Defining the expected return and variance

def expectedReturn(R, pr, k, d, k0, C): 
    retSum = 0.0
    for i in range(len(R)):
        val = R[i]*p[i]*k[i] - d[i]*p[i]*abs(k[i]-k0[i])
        retSum = retSum + val
    expRet = retSum/C
    return expRet

def variance(sigma, pr, k, C): 
    var = 0.0
    for i in range(len(pr)):
        for j in range(len(pr)):
            var = var + (sigma[i][j]*pr[i]*k[i]*pr[j]*k[j])/(C**2)
    return var

In [4]:
# Defining the repair process : 
def repairProcess(k0 , x, nS, nB):
    m = np.array(k0)
    for i in range(len(x)):
        d = random.randint(0,1)
        if x[i] >= k0[i] :
            d = d + (x[i] - k0[i])/nB[i]
            m[i] = m[i] + d*nB[i]
        else :
            d = d + (k0[i] - x[0])/nS[i]
            m[i] = m[i] - d*nS[i]
    return m 

In [5]:
def fitnessFunction(lambd, expRet, var, omega) :
    return (((-1)*lambd*expRet) + ((1-lambd)*omega*var))

In [6]:
def cxSimulatedBinary(ind1, ind2, eta):
    """Executes a simulated binary crossover that modify in-place the input
    individuals. The simulated binary crossover expects :term:`sequence`
    individuals of floating point numbers.
    :param ind1: The first individual participating in the crossover.
    :param ind2: The second individual participating in the crossover.
    :param eta: Crowding degree of the crossover. A high eta will produce
                children resembling to their parents, while a small eta will
                produce solutions much more different.
    :returns: A tuple of two individuals.
    This function uses the :func:`~random.random` function from the python base
    :mod:`random` module.
    """
    for i, (x1, x2) in enumerate(zip(ind1, ind2)):
        rand = random.random()
        if rand <= 0.5:
            beta = 2. * rand
        else:
            beta = 1. / (2. * (1. - rand))
        beta **= 1. / (eta + 1.)
        ind1[i] = 0.5 * (((1 + beta) * x1) + ((1 - beta) * x2))
        ind2[i] = 0.5 * (((1 - beta) * x1) + ((1 + beta) * x2))

    return ind1, ind2


def PMmutation(individual, indpb):
    """Shuffle the attributes of the input individual and return the mutant.
    The *individual* is expected to be a :term:`sequence`. The *indpb* argument is the
    probability of each attribute to be moved. Usually this mutation is applied on
    vector of indices.
    :param individual: Individual to be mutated.
    :param indpb: Independent probability for each attribute to be exchanged to
                  another position.
    :returns: A tuple of one individual.
    This function uses the :func:`~random.random` and :func:`~random.randint`
    functions from the python base :mod:`random` module.
    """
    size = len(individual)
    for i in range(size):
        if random.random() < indpb:
            swap_indx = random.randint(0, size - 2)
            if swap_indx >= i:
                swap_indx += 1
            individual[i], individual[swap_indx] = \
                individual[swap_indx], individual[i]

    return individual

In [7]:
# Helper Functions
def calculateC(p, k):
    C = 0.0
    for i in range(len(p)):
        C = C + p[i]*k[i]
    return C
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [8]:
# we extract all the Open Price from the dataset 
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.csv"))
retu = np.array([[]]) 
# loop over the list of csv files
for f in csv_files:
    # read the csv file
    df = pd.read_csv(f)
    print('Code of the Asset:', f[58:])
    retEach = np.array([])
    df = df[['Date','Open Price']][:101]
    for i in range(1,101) :
        val = (df['Open Price'][i] - df['Open Price'][i-1])/ df['Open Price'][i-1]
        retEach = np.append(retEach, val)
    retu = np.append(retu, retEach)

Code of the Asset: 502168.csv
Code of the Asset: 500575.csv
Code of the Asset: 526441.csv
Code of the Asset: 500285.csv
Code of the Asset: 513430.csv
Code of the Asset: 543280.csv
Code of the Asset: 533269.csv
Code of the Asset: 500870.csv
Code of the Asset: 507155.csv
Code of the Asset: 531508.csv
Code of the Asset: 532725.csv
Code of the Asset: 532234.csv
Code of the Asset: 511509.csv
Code of the Asset: 504093.csv
Code of the Asset: 519156.csv
Code of the Asset: 541729.csv
Code of the Asset: 500463.csv
Code of the Asset: 532493.csv
Code of the Asset: 523618.csv
Code of the Asset: 524667.csv
Code of the Asset: 500112.csv
Code of the Asset: 507685.csv
Code of the Asset: 509966.csv
Code of the Asset: 533278.csv
Code of the Asset: 532475.csv
Code of the Asset: 526721.csv
Code of the Asset: 540750.csv
Code of the Asset: 532454.csv
Code of the Asset: 517354.csv
Code of the Asset: 500875.csv


In [9]:
ret1 = retu.reshape(30,100)
covarianceMatrix = np.cov(ret1)

In [10]:
# k0 will be given and it is the existing portfolio in hand, 
# we have to find out value of the vector k
k0 = np.random.randint(1,16,size=(30))
nS = np.random.randint(1,4,size=(30))
nB = np.random.randint(1,4,size=(30))
lambd = np.array([0.0,0.2,0.4,0.6,0.8,1.0])
omega = 100

covarianceMatrix = np.cov(ret1)
R = np.array([])
for i in range(ret1.shape[0]):
    R = np.append(R, np.mean(ret1[i]))
    
p = np.random.uniform(low=0.5, high=13.3, size=(30,))
d = np.full(30,0.01)

In [13]:
gener = 300
for i in range(gener):
    print("Doing for epoch : ", i)
    # Creating the population
    population = np.array([[]])
    for i in range(pop_size):
        member = np.array([])
        for j in range(len(k0)):
            val = random.randint(0,10)
            member = np.append(member,val)
        population = np.append(population,member)
    population = population.reshape(pop_size, len(k0))

    for run in range(gener):
        # SBX Crossover
        loop_found = {}
        for i in range(pop_size):
            if loop_found.get(i):
                continue
            else :
                loop_found[i] = True
                next_ind = random.randint(i+1, pop_size-1)
                while loop_found.get(next_ind):
                    next_ind = random.randint(i+1, pop_size-1)
                loop_found[next_ind] = True
                population = np.append(population, [population[i]], axis=0)
                population = np.append(population, [population[next_ind]], axis=0)
                cxSimulatedBinary(population[i],population[next_ind] , P_c)

        # Mutation
        for i in range(len(population)):
            PMmutation(population[i], 1/len(population[0]))

        # Doing the repair process : 
        for i in range(len(population)):
            population[i] = repairProcess(k0,population[i],nS,nB)

        fitness_values = {}
        for i in range(len(population)):
            C = calculateC(p, population[i])
            expRet = expectedReturn(R, p, population[i], d, k0, C)
            var = variance(covarianceMatrix, p, population[i], C)
            fitness = fitnessFunction(lambd[1], expRet, var, omega)
            fitness_values[i] = abs(fitness)

        new_dict = {k: v for k, v in sorted(fitness_values.items(), key=lambda item: item[1])}
        n_items = take(pop_size, new_dict.items())
        next_gen = np.array([[]])
        for i in range(len(n_items)):
            next_gen = np.append(next_gen, population[n_items[i][0]])
        next_gen = next_gen.reshape(pop_size, len(k0))
        population = next_gen

Doing for epoch :  0
Doing for epoch :  1
Doing for epoch :  2
Doing for epoch :  3
Doing for epoch :  4
Doing for epoch :  5
Doing for epoch :  6
Doing for epoch :  7
Doing for epoch :  8
Doing for epoch :  9
Doing for epoch :  10
Doing for epoch :  11
Doing for epoch :  12
Doing for epoch :  13
Doing for epoch :  14
Doing for epoch :  15
Doing for epoch :  16
Doing for epoch :  17
Doing for epoch :  18
Doing for epoch :  19
Doing for epoch :  20
Doing for epoch :  21
Doing for epoch :  22
Doing for epoch :  23
Doing for epoch :  24
Doing for epoch :  25
Doing for epoch :  26
Doing for epoch :  27
Doing for epoch :  28
Doing for epoch :  29
Doing for epoch :  30
Doing for epoch :  31
Doing for epoch :  32
Doing for epoch :  33
Doing for epoch :  34
Doing for epoch :  35
Doing for epoch :  36
Doing for epoch :  37
Doing for epoch :  38
Doing for epoch :  39
Doing for epoch :  40
Doing for epoch :  41
Doing for epoch :  42
Doing for epoch :  43
Doing for epoch :  44
Doing for epoch :  4

In [17]:
minFitness = 999999
idx = 0
for i in range(len(population)):
    C = calculateC(p, population[i])
    expRet = expectedReturn(R, p, population[i], d, k0, C)
    var = variance(covarianceMatrix, p, population[i], C)
    fitness = fitnessFunction(lambd[1], expRet, var, omega)
    if fitness < minFitness:
        idx = i
        minFitness = fitness
print(minFitness, " at index ", idx)

0.008901435444217622  at index  0


In [18]:
print("Optimal distribution of k is : ")
print(population[idx])

Optimal distribution of k is : 
[-182834. -182835. -182834.  174566. -182837. -182834. -182834. -182834.
 -182834. -182835. -182836. 1120807. -182834. -182834. -182834. -182837.
    4046. -182836. -182834. -182834. -182837. -182834. -182834. -182835.
 -182834. -182834. -182836. -182837. -182835. -182835.]
